### Determining the Optimum Number of Channels with Correlated Neural Signals

This Jupyter notebook contains designed to dynamically assess and optimize the number of recording sites within a probe's columns based on significant correlations in neural recording data. The approach adjusts for variability in experimental conditions, dataset properties, brain areas, and probe designs. By calculating the average correlation among recording sites and comparing it against a predefined threshold, the script determines the optimal number of channels that exhibit similar neural activity patterns. This method provides a flexible and accurate way to tailor probe configuration to various experimental setups, enhancing the reliability of neural data analysis.

In [ ]:
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

In [ ]:
# defining related functions

def smooth_neural_data(signal, window_size=10):
    """Apply a moving average to make the neural activity pattern smoother."""
    return np.convolve(signal, np.ones(window_size)/window_size, mode='same')

# Function to calculate average correlation for each recording site within a column
def average_correlation(data, num_sites, threshold):
    avg_correlations = []
    for i in range(num_sites):
        correlations = []
        for j in range(num_sites):
            if i != j:
                corr, _ = pearsonr(data[i], data[j])
                correlations.append(corr)
        avg_corr = np.mean(correlations) if correlations else 0
        avg_correlations.append(avg_corr)
    significant_sites = sum(corr > threshold for corr in avg_correlations)
    return significant_sites



In [ ]:
# Parameters for the initial window of neural data
np.random.seed(42)
num_columns = 3
max_sites_per_column = 50
data_length = 1000  
correlation_threshold = 0.85

In [ ]:
def generate_neural_activity(rate, length, shared_activity=None, amplitude=1):
    """Generates a random neural activity pattern for a neuron, including shared activity."""
    times = np.random.rand(int(rate * length)) * length  # Random neural activity times
    neural_activity = np.zeros(length)
    neural_activity[np.floor(times).astype(int)] = amplitude  # Set activity
    
    if shared_activity is not None:
        neural_activity += shared_activity  # Add shared activity to simulate more correlation

    return neural_activity


initial_neural_data = []
for col in range(num_columns):
    shared_activity = generate_neural_activity(0.05, data_length, amplitude=5)  # Shared component
    column_data = [smooth_neural_data(generate_neural_activity(0.05, data_length, shared_activity, np.random.rand()*5))
                   for _ in range(max_sites_per_column)]
    initial_neural_data.extend(column_data)

initial_neural_data = np.array(initial_neural_data)

In [ ]:
# Analyze each column for the initial data window
optimal_k = []
for col in range(num_columns):
    col_data = initial_neural_data[col * max_sites_per_column:(col + 1) * max_sites_per_column]
    significant_sites = average_correlation(col_data, max_sites_per_column, correlation_threshold)
    optimal_k.append(significant_sites)

print("Number of recording sites per column with correlations > threshold:", optimal_k)
